# Lab. 14 - Raporty i wizualizacja danych

## Kontrolki interaktywne w Jupyter Notebook
czyli jak korzystać z interaktywnych widgetów IPython, aby usprawnić eksplorację i analizę danych.

W eksploracji danych mało efektywne i zakłócające płynność analizy danych jest wielokrotne uruchamianie tej samej komórki, za każdym razem nieznacznie zmieniając parametry wejściowe, np. wybierając inną wartość funkcji, inne zakresy dat do analizy, czy motyw wizualizacji plotly.

Jednym z rozwiązań tego problemu są interaktywne kontrolki umożliwiające zmianę danych wejściowych bez konieczności przepisywania lub ponownego uruchamiania kodu. Mowa o **IPython widgets** (z biblioteki ipywidgets), które można zbudować za pomocą jednej linii kodu. Biblioteka pozwala nam przekształcić statyczne dokumenty Jupyter Notebook w interaktywne pulpity, idealne do eksploracji i wizualizacji danych.

1. Instalacja: 
```cmd
pip install ipywidgets 
pip install pyarrow
pip install fastparquet
pip install chart_studio
```

1. Aktywacja widgetów dla Jupyter Notebook: 
```cmd
jupyter nbextension enable --py widgetsnbextension
```
2. Import: 
```python
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
```

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import chart_studio.plotly as py

In [ ]:
import pandas as pd
import numpy as np

Pokaż wszystkie dane wyjściowe komórek

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Podstawowe widgety
Przed rozpoczęciem wykonywania głównego zadania, proszę zapoznać się z dokumentacją dostępną [tutaj](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html). Zawiera ona przykłady najprostszych widgetów. Dostępna jest również wersja interaktywna, proszę ją wykonać. 

[Tutaj](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) znajduje się lista widgetów. Niech to będzie punkt odniesienia w trakcie pracy z wykorzystaniem widgetów. \
Cała dokumentacja jest dostępna [tutaj](https://ipywidgets.readthedocs.io/en/stable/user_guide.html).

## Wczytanie danych
Dane pochodzą z [repozytorium Willa Koehrsena](https://github.com/WillKoehrsen/Data-Analysis) i zawierają statystyki dotyczące jego artykułów.

Więcej informacji znajdziesz [tutaj](https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6).

In [ ]:
data = pd.read_parquet('https://github.com/WillKoehrsen/Data-Analysis/blob/master/medium/data/medium_data_2019_01_26?raw=true')
data.head()

Proszę zapoznać się z danymi (.columns, .describe(), .info()).

Stwórz nowy DataFrame, z kolumnami: title, tags, published_date, publication, reads, views, word_count, claps, fans, read_time.

## Wyświetlanie danych

Za pomocą *.loc* wyświetl artykuły, które zostały przeczytane więcej niż 1000 razy.

Teraz za pomocą *.loc* wyświetl artykuły, które zostały przeczytane więcej niż 500 razy.

To najprostszy przykład pokazujący, że interaktywna zmiana parametrów jest potrzebna, by usprawnić proces analizy danych. \
Można to zrobić na przykład za pomocą specjalnej metody, z dekoratorem *@interact*:

In [ ]:
from IPython.display import display, HTML

In [ ]:
@interact
def show_articles_more_than(column='claps', x=5000):
    display(HTML(f'<h3>Showing articles with more than {x} {column}<h3>'))
    display(df.loc[df[column] > x])

Dokumentacja [Interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html). Tłumaczy m. in., w jaki sposób parametry funkcji są mapowane do widgetów.

Zauważ, że dekorator *@interact* automatycznie wywnioskował, że potrzebujemy pola tekstowego dla kolumny i suwaka int dla x. 

Gdy potrzebujemy wymusić pewne ograniczenia interakcji, możemy ustawić dodatkowe opcje tworzonej funkcji, takie jak *dropdown* czy granice dla wielkości numerycznych - format to (start, stop, krok):

In [ ]:
@interact
def show_titles_more_than(column=list(df.select_dtypes('number').columns), 
                          x=(1000, 5000, 100)):
    display(HTML(f'<h3>Showing articles with more than {x} {column}<h3>'))
    display(df.loc[df[column] > x])

## Dataframe explorer

Stwórz funkcję z dekoratorem *@interact*, żeby szybko znajdować korelację między dwoma wybranymi kolumnami.

Stwórz funkcję z dekoratorem *@interact*, żeby wywołać funkcję *describe()* dla wybranej kolumny.

## Widgety dla wykresów
Widgety interaktywne są szczególnie przydatne przy wybieraniu danych do wykresu. W tym wypadku również możemy użyć tego samego dekoratora *@interact*, z funkcjami wizualizującymi nasze dane.

**Uwaga.** Obiekt DataFrame nie ma metody iplot, jeśli nie jest połączony z plotly. Potrzebujemy **cufflinks**, aby połączyć pandas z plotly i dodać metodę iplot.

Dodatkowo, aby uniknąć uwierzytelniania, potrzebujemy trybu offline.

In [ ]:
import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

In [ ]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:]):
    df.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), title=f'{y.title()} vs {x.title()}')

Dodaj więcej opcji do funkcji scatter_plot:
1. Parametr theme, korzystając z tych dostępnych w cf.themes.THEMES.keys()
2. Parametr colorscale, korzystając z tych dostępnych w cf.colors.\_scales\_names.keys())

Do funkcji scatter_plot dodaj parametr categories, grupujący nasze dane. Przetestuj jego działanie.

In [ ]:
df['binned_read_time'] = pd.cut(df['read_time'], bins=range(0, 56, 5))
df['binned_read_time'] = df['binned_read_time'].astype(str)

df['binned_word_count'] = pd.cut(df['word_count'], bins=range(0, 100001, 1000))
df['binned_word_count'] = df['binned_word_count'].astype(str)

categories=['binned_read_time', 'binned_word_count', 'publication']

Być może zauważyłeś, że aktualizacja wykresu przebiegała powoli. W takim przypadku możesz użyć dekoratora *@interact_manual*, który dostarcza przycisku do aktualizacji. 

Sprawdź działanie widgetu z dekoratorem *@interact_manual*.

In [ ]:
from ipywidgets import interact_manual

## Własne widgety
Aby skorzystać jeszcze więcej z biblioteki ipywidgets, możemy sami tworzyć widgety i używać ich w funkcji interakcji.

Stwórz własny widget. Napisz funkcję stats_for_article_published_between, która pobiera datę początkową i końcową, oraz wyświetla statystyki dla wszystkich artykułów opublikowanych między nimi.

In [ ]:
df.set_index('published_date', inplace=True)

In [ ]:
def stats_for_article_published_between(start_date, end_date):
    ...
    print(f'You published {num_articles} articles between {start_date.date()} and {end_date.date()}.')
    print(f'These articles totalled {total_words:,} words and {total_read_time/60:.2f} hours to read.')

Za pomocą następującego kodu funkcja staje się interaktywna:

In [ ]:
_ = interact(stats_for_article_published_between,
             start_date=widgets.DatePicker(value=pd.to_datetime('2018-01-01')),
             end_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

Napisz funkcję plot_up_to, aby narysować wykres kumulatywnej sumy wartości wybranej kolumny, do wybranego dnia.  

Użyj Dropdown i DatePicker w funkcji *interact*.

In [ ]:
def plot_up_to(column, date):
    ...

## Przeglądanie zdjęć

Stwórz funkcję z dekoratorem *@interact*, żeby przeglądać zdjęcia znajdujące się w wybranym folderze. Folder z 3-5 zdjęciami również umieść na repozytorium.

In [ ]:
import os
from IPython.display import Image

## Przeglądanie plików

Stwórz funkcję z dekoratorem *@interact*, żeby przeglądać pliki znajdujące się w wybranych folderach.
Skorzystaj z następujących (przykładowych) opcji komendy *ls*: **ls -a -t -r -l -h**. Więcej informacji znajduje się [tutaj](https://www.rapidtables.com/code/linux/ls.html).

In [ ]:
import subprocess

## Zależne widgety
Jeśli chcemy opcje jednego widgetu uzależnić od wartości innego widgetu, używamy metody *observe*. 

Wykorzystaj metodę *observe*, żeby zmienić funkcję przeglądania zdjęć tak, by móc wybierać zarówno ścieżkę, jak i obraz do wyświetlenia. Drugi folder z 3-5 zdjęciami również umieść na repozytorium.

In [ ]:
directory = widgets.Dropdown(options=[...])
images = widgets.Dropdown(options=os.listdir(directory.value))

def update_images(*args):
    images.options = ...

directory.observe(update_images, 'value')

def show_images(fdir, file):
    ...

_ = interact(...)

Możemy również przypisać zmienną do outputu funkcji *interact*, a następnie ponownie użyć widgetu. Może mieć to jednak niezamierzone skutki!

Teraz zmiana wartości w jednej lokalizacji zmienia ją w obu miejscach! Może to być drobna niedogodność, ale zaletą jest to, że możemy ponownie wykorzystać interaktywny element.

In [ ]:
dependent_widget = interact(show_images, fdir=directory, file=images)

In [ ]:
dependent_widget.widget

## Wnioski